In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Importing libraries

In [ ]:
!pip install transformers

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import random
import shutil
import sys
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import os
from transformers import AutoModel

# Setting seed for reproducibility

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
set_seed(43)

# Downloading dataset

In [ ]:
from requests import get as rget

# Update URL to raw CSV file
url = "https://raw.githubusercontent.com/Fal186/Mapping-web3/refs/heads/main/dataset/web3_Stage3_communicative_intent.csv"

res = rget(url)
with open('file.csv', 'wb+') as f:
    f.write(res.content)

train_df = pd.read_csv('file.csv')

In [ ]:
train_df.info()

In [ ]:
possible_labels = train_df.label.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

In [ ]:
train_df['label'] = train_df.label.replace(label_dict)

In [ ]:
train_df.head()

In [ ]:
# --- CONFIGURATION FOR MULTI-CLASS ---
IS_MULTI_CLASS = True

# Create your target list from the unique labels found in your dictionary
target_list = list(label_dict.keys())
NUM_LABELS_STAGE = len(target_list)

print(f"Switching to Multi-Class with {NUM_LABELS_STAGE} categories: {target_list}")

# Setting Base hyperparameters

In [ ]:
BATCH_SIZE = 16
EPOCHS = 5
PATIENCE = 2
LEARNING_RATE = 2e-5
GLOBAL_SEED = 43

##Implementing Focal loss


In [ ]:
#FOCAL LOSS IMPLEMENTATION
class FocalLoss(nn.Module):
    """
    Implements Focal Loss
    """
    def __init__(self, alpha=0.25, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        # Alpha (α) balances positive/negative samples; Gamma (γ) focuses on hard examples
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # 1. Calculate BCE Loss (Logits are used for numerical stability)
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')

        # 2. Convert logits to probabilities (p)
        p = torch.sigmoid(inputs)

        # Calculate p_t (probability of the *true* class)
        # If target is 1 (positive), p_t is p. If target is 0 (negative), p_t is 1-p.
        p_t = p * targets + (1 - p) * (1 - targets)

        # 3. Calculate Alpha Weighting (alpha_t)
        # If target is 1 (positive), alpha_t is alpha. If target is 0 (negative), alpha_t is 1-alpha.
        alpha_factor = targets * self.alpha + (1 - targets) * (1 - self.alpha)

        # 4. Calculate Modulating Factor: ((1 - p_t) ^ gamma)
        # This is the core term that suppresses loss for easy, well-classified examples.
        modulating_factor = (1.0 - p_t) ** self.gamma

        # 5. Combine and Calculate Final Loss
        Focal_Loss = alpha_factor * modulating_factor * BCE_loss

        if self.reduction == 'mean':
            return Focal_Loss.mean()
        elif self.reduction == 'sum':
            return Focal_Loss.sum()
        else:
            return Focal_Loss

#Modular Model Switching

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn

# --- MODEL MAPPING CONSTANTS ---
MODEL_MAPPING = {
    'BERT': 'bert-base-uncased',
    'ROBERTA': 'roberta-base',
    'CRYPTOBERT': 'ElKulako/cryptobert'
}

# --- UNIFIED MODEL CLASS ---
class FlexibleTransformerModel(nn.Module):
    """
    Unified class to load BERT, RoBERTa, or CryptoBERT and apply classifier head.
    """
    def __init__(self, model_name_or_path, num_labels):
        super(FlexibleTransformerModel, self).__init__()

        self.base_model = AutoModel.from_pretrained(model_name_or_path, return_dict=True)
        hidden_size = self.base_model.config.hidden_size

        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attn_mask, token_type_ids=None):

        output = self.base_model(
            input_ids=input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids # Handles models that don't use token_type_ids
        )

        # Use the hidden state of the [CLS] token (first token)
        cls_output = output.last_hidden_state[:, 0, :]

        output_dropout = self.dropout(cls_output)
        output = self.linear(output_dropout)

        return output

In [ ]:
# --- STEP 1: Define YOUR MODEL ---

# Define the number of output labels for your current classification task
# Exapmle for Multli Label
#NUM_LABELS_STAGE = 5

# Example for Multi-Class
NUM_LABELS_STAGE = 3

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cuda'

In [ ]:
#Select the model
CURRENT_MODEL_KEY = 'ROBERTA'  # <--- Change this string to 'BERT' or 'ROBERTA'

# Preparing the tokenizer

In [ ]:
#Set Max Lenght, max 512
MAX_LEN = 512

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load the Tokenizer based on the model
def load_model_and_tokenizer(model_key, num_labels=5, device='cuda'):
    """ Loads the model and tokenizer based on the simple model key. """
    model_path = MODEL_MAPPING.get(model_key)
    if not model_path:
        raise ValueError(f"Unknown model key: {model_key}. Use one of {list(MODEL_MAPPING.keys())}")

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = FlexibleTransformerModel(model_path, num_labels)
    model.to(device)

    return model, tokenizer

In [ ]:
# Call the helper function to load the model and its matching tokenizer
model, tokenizer = load_model_and_tokenizer(
    model_key=CURRENT_MODEL_KEY,
    num_labels=NUM_LABELS_STAGE,
    device=device
)

print(f"Successfully loaded {CURRENT_MODEL_KEY} and its tokenizer.")

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.text = df['text']
        self.targets = self.df['label']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)
        }

# Splitting & Tokenizing Dataset

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import DataLoader, random_split, Subset
import torch
from typing import Tuple

#1. SEPARATE THE FINAL 10% TEST SET

ALL_DATA_DF = train_df

# Split the data into 90% Working Set (for k-fold) and 10% Final Test Set (held-out)
FULL_TRAIN_VAL_DF, TEST_DF = train_test_split(
    ALL_DATA_DF,
    test_size=0.10,
    random_state=GLOBAL_SEED
)

# Reset indices for clean subsetting later
FULL_TRAIN_VAL_DF = FULL_TRAIN_VAL_DF.reset_index(drop=True)
TEST_DF = TEST_DF.reset_index(drop=True)

print(f"Total Data: {len(ALL_DATA_DF)}")
print(f"90% Working Set for K-Fold: {len(FULL_TRAIN_VAL_DF)} samples.")
print(f"10% Final Test Set (Held Out): {len(TEST_DF)} samples.")


# --- 2. DEFINE THE DATASET CREATION FUNCTION ---

def create_datasets_and_loaders(
    working_df: pd.DataFrame,
    test_df: pd.DataFrame,
    tokenizer,
    max_len: int,
    batch_size: int,
    subset_for_tuning: bool = True,
    seed: int = 43 # Added seed parameter with a default value
) -> Tuple[DataLoader, DataLoader, DataLoader]:

    # Create Custom Datasets from DataFrames
    full_train_val_dataset = CustomDataset(working_df, tokenizer, max_len)
    test_dataset = CustomDataset(test_df, tokenizer, max_len)

    # --- 50% SUBSET LOGIC ---
    if subset_for_tuning:
        total_size = len(full_train_val_dataset)
        subset_size = total_size // 2
        remainder_size = total_size - subset_size

        # Ensure reproducible split for the subset
        generator = torch.Generator().manual_seed(seed)

        # Select the 50% subset for quick tuning
        train_val_working_set, _ = random_split(
            full_train_val_dataset,
            [subset_size, remainder_size],
            generator=generator
        )
        print(f"NOTE: Using 50% Subset for Hyperparameter Tuning: {len(train_val_working_set)} samples.")
    else:
        # Use the full 90% working set for k-fold (or single full split)
        train_val_working_set = full_train_val_dataset
        print(f"Using Full Working Set: {len(train_val_working_set)} samples.")

    #Split the Working Set into Train and Validation (80/20)

    # We must split the working set into a train/validation split now for the quick search
    # This split is NOT needed for the final K-Fold runs, but is necessary for the initial tuning.

    generator = torch.Generator().manual_seed(seed + 1)

    train_size = int(0.8 * len(train_val_working_set))
    val_size = len(train_val_working_set) - train_size

    train_dataset, val_dataset = random_split(
        train_val_working_set,
        [train_size, val_size],
        generator=generator
    )

    print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

    # --- Create DataLoaders ---
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    return train_data_loader, val_data_loader, test_data_loader


In [ ]:
#DISTRIBUTION OF THE 90% WORKING SET (K-FOLD DATA)
print("==================================================================")
print("             LABEL DISTRIBUTION CHECK (POST-SPLIT)                ")
print("==================================================================")

# Counts for the 90% Working Set (will be used for K-Fold training)
working_counts = FULL_TRAIN_VAL_DF['label'].sum(axis=0)
working_counts_percentage = (working_counts / len(FULL_TRAIN_VAL_DF)) * 100

print(f"### 90% Working Set ({len(FULL_TRAIN_VAL_DF)} Samples) ###")
print("Absolute Counts:\n", working_counts)
print("\nPercentage:\n", working_counts_percentage.round(2))

#DISTRIBUTION OF THE 10% FINAL TEST SET (HELD-OUT DATA)

# Counts for the 10% Final Test Set (will be used for final evaluation)
test_counts = TEST_DF['label'].sum(axis=0)
test_counts_percentage = (test_counts / len(TEST_DF)) * 100

print(f"\n### 10% Final Test Set ({len(TEST_DF)} Samples) ###")
print("Absolute Counts:\n", test_counts)
print("\nPercentage:\n", test_counts_percentage.round(2))
print("------------------------------------------------------------------")

In [ ]:
#OVERLAP CHECK (Working Set vs. Final Test Set)

# We only need to check for overlap between the two main blocks
working_texts = set(FULL_TRAIN_VAL_DF['text'])
test_texts = set(TEST_DF['text'])

# The intersection gives the number of samples present in both sets
overlap = working_texts.intersection(test_texts)
print(f"\nCRITICAL CHECK: Overlapping samples between Working Set and Final Test Set: {len(overlap)}")

if len(overlap) > 0:
    print("WARNING: Overlap detected! This must be addressed before running K-Fold.")
else:
    print("SUCCESS: No overlap detected between the Training/Validation and Test Sets.")
print("==================================================================")

In [ ]:
print("--- Data Shapes Check (Post 90/10 Split) ---")

# 1. Check the 90% Working Set (The data for K-Fold)
print(f"FULL_TRAIN_VAL_DF Shape (90% Working Set): {FULL_TRAIN_VAL_DF.shape}")

# 2. Check the 10% Final Test Set (Held-Out Data)
print(f"TEST_DF Shape (10% Final Test Set): {TEST_DF.shape}")

print("---------------------------------------------")

In [ ]:
# If you still want to check the head of the Test Set
print("\nFULL_TRAIN_VAL_DF Head (Final Working Data):")
print(FULL_TRAIN_VAL_DF.head())

In [ ]:
# If you still want to check the head of the Test Set
print("\nTEST_DF Head (Final Test Data):")
print(TEST_DF.head())

In [ ]:
# Create the CustomDataset for each set
full_train_val_dataset = CustomDataset(FULL_TRAIN_VAL_DF, tokenizer, MAX_LEN)
test_dataset = CustomDataset(TEST_DF, tokenizer, MAX_LEN)

print("CustomDatasets created for the Working Set and the Final Test Set.")

In [ ]:
len(full_train_val_dataset)

# Additional functions for loading and saving checkpoints

In [ ]:
import torch
import shutil
import numpy as np

import torch
import shutil
import numpy as np

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # FIX: weights_only=False allows loading checkpoints containing numpy/metadata
    checkpoint = torch.load(checkpoint_fpath, map_location=device, weights_only=False)

    # Initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])

    # Initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])

    # Safely get metadata using .get() to prevent crashes
    valid_loss_min = checkpoint.get('valid_loss_min', np.inf)
    epoch = checkpoint.get('epoch', 1)
    macro_f1 = checkpoint.get('macro_f1', 0.0) # Used for skip logic

    # Return model, optimizer, epoch value, min validation loss, and f1 score
    return model, optimizer, epoch, valid_loss_min, macro_f1

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, checkpoint_path)

    # if it is a best model, copy that checkpoint file to best path
    if is_best:
        shutil.copyfile(checkpoint_path, best_model_path)
        print(f"🥇 Best model saved to: {best_model_path}")

# Training the Model

Setting Up the Loss Function and Optimizer

In [ ]:
# Initialize the FocalLoss instance once (gamma=2, alpha=0.25 are good starting points)
FOCAL_LOSS_CRITERION = FocalLoss(alpha=0.25, gamma=2.0)
def focal_loss_fn(outputs, targets):
    return FOCAL_LOSS_CRITERION(outputs, targets)

def cross_entropy_loss_fn(outputs, targets):
    # If targets is 1D (shape: [batch_size]), it's already class indices.
    # If targets is 2D (shape: [batch_size, num_labels]), we need argmax.
    if len(targets.shape) == 1:
        return nn.CrossEntropyLoss()(outputs, targets.long())
    else:
        return nn.CrossEntropyLoss()(outputs, targets.argmax(dim=1).long())

# Define a flag to switch loss
IS_MULTI_CLASS = True

def get_current_loss_fn(is_multi_class):
    if is_multi_class:
        # For Multi-Class, use Cross-Entropy
        return cross_entropy_loss_fn
    else:
        # For Multi-Label, use Focal Loss
        return focal_loss_fn

optimizer = torch.optim.AdamW(params = model.parameters(), lr=LEARNING_RATE)

Training and Validation Loop with Early Stopping

In [ ]:
import torch.nn as nn
import torch
import numpy as np

# NOTE: Ensure loss_fn, device, and the save_ckp function are defined elsewhere

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    # Always save the full state to the general checkpoint path
    torch.save(state, checkpoint_path)

    # If it is the best model, save the FULL STATE to the best model path
    if is_best:
        torch.save(state, best_model_path)
        print(f"Best model saved to {best_model_path}")

def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path, patience,
                IS_MULTI_CLASS): # <-- Corrected function signature

    # Initialize tracker for minimum validation loss
    valid_loss_min = np.inf
    no_improve = 0

    # 1. INITIALIZE LISTS to hold the best validation data from the BEST epoch
    best_val_targets_data = []
    best_val_outputs_data = []

    for epoch in range(1, n_epochs + 1):
        train_loss = 0
        valid_loss = 0

        # --- TRAINING LOOP (Uses loss_fn which should be Focal Loss) ---
        model.train()
        print(f'############# Epoch {epoch}: Training Start   #############')

        # --- FIX: The loop over batches MUST be indented here ---
        for batch_idx, data in enumerate(training_loader):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            # FIX 1: Preserve original long dtype for targets
            targets = data['targets'].to(device)

            outputs = model(ids, mask, token_type_ids)

           #DYNAMIC LOSS FUNCTION CALL
            current_loss_fn = get_current_loss_fn(IS_MULTI_CLASS)

            optimizer.zero_grad()
            # Ensure targets are long for CrossEntropyLoss if it's a 1D tensor of indices
            # current_loss_fn already handles .long() internally if targets is 1D
            if IS_MULTI_CLASS: # Pass targets as long for multi-class
                loss = current_loss_fn(outputs, targets.long())
            else: # Pass targets as float for multi-label
                loss = current_loss_fn(outputs, targets.float())

            loss.backward()
            optimizer.step()

            train_loss += ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        # -----------------------------------------------------------------

        print(f'############# Epoch {epoch}: Training End     #############')

        # --- VALIDATION LOOP (Collecting data for M1) ---
        print(f'############# Epoch {epoch}: Validation Start   #############')
        model.eval()

        # Initialize collectors for the current epoch's data
        current_epoch_val_outputs = []
        current_epoch_val_targets = []

        with torch.no_grad():
            # --- FIX: The loop over validation batches MUST be indented here ---
            for batch_idx, data in enumerate(validation_loader, 0):
                ids = data['input_ids'].to(device, dtype=torch.long)
                mask = data['attention_mask'].to(device, dtype=torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
                # FIX 1: Preserve original long dtype for targets
                targets = data['targets'].to(device)
                outputs = model(ids, mask, token_type_ids)

                # Use correct Validation Loss function ---
                if IS_MULTI_CLASS:
                    # FIX 2: Targets are already 1D long class indices, pass directly
                    loss = nn.CrossEntropyLoss()(outputs, targets.long())
                else:
                    # Multi-Label: Use standard BCE loss for validation tracking
                    loss = nn.BCEWithLogitsLoss()(outputs, targets.float())

                valid_loss += ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))

                # Use correct Activation function (Sigmoid vs Softmax) ---
                if IS_MULTI_CLASS:
                    # Multi-Class: Use Softmax
                    probabilities = torch.softmax(outputs, dim=1)
                    # Convert 1D targets to one-hot for consistency with multi-label output format for metrics
                    # This is for internal metric calculation, not the loss itself.
                    # Number of classes needed for one_hot
                    num_classes = outputs.shape[1]
                    one_hot_targets = F.one_hot(targets.long(), num_classes=num_classes).float()
                    current_epoch_val_targets.extend(one_hot_targets.cpu().detach().numpy().tolist())
                else:
                    # Multi-Label: Use Sigmoid (for Adaptive Thresholds)
                    probabilities = torch.sigmoid(outputs)
                    current_epoch_val_targets.extend(targets.cpu().detach().numpy().tolist())

                current_epoch_val_outputs.extend(probabilities.cpu().detach().numpy().tolist())

        # -------------------------------------------------

        train_loss /= len(training_loader)
        valid_loss /= len(validation_loader)
        print(f'Epoch: {epoch} \tAverage Training Loss: {train_loss:.6f} \tAverage Validation Loss: {valid_loss:.6f}')

        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        # Save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model ...')

            # 3. SAVE THE DATA from the current best epoch
            best_val_targets_data = current_epoch_val_targets
            best_val_outputs_data = current_epoch_val_outputs

            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss
            no_improve = 0
        else:
            no_improve += 1
            print(f"No improvement in validation loss for {no_improve} epoch(s).")

        # Early stopping
        if no_improve >= patience:
            print("Early stopping due to no improvement in validation loss.")
            break

    # 4. RETURN the model and the data from the best epoch
    return model, np.array(best_val_targets_data), np.array(best_val_outputs_data)

In [ ]:
import os

# Define the base directories clearly
CKPT_DIR = "/content/gdrive/MyDrive/ckpt_path_multi_class_roberta"
BEST_MODEL_DIR = "/content/gdrive/MyDrive/Best_Model_multi_class_roberta"

# Ensure directories exist
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(BEST_MODEL_DIR, exist_ok=True)

#Implementing Adaptive Threshold

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def calculate_adaptive_thresholds(y_true, y_pred_proba):
    """
    Calculates the optimal threshold for each class by maximizing the F1-score
    on the validation set.

    Args:
        y_true (np.array): True binary labels (e.g., targets from validation set).
        y_pred_proba (np.array): Predicted probabilities (sigmoid output from model).

    Returns:
        np.array: A 1D array of optimal thresholds, one for each class.
    """

    # Get the number of classes (columns)
    num_classes = y_true.shape[1]
    optimal_thresholds = []

    # Iterate through each class (column) independently
    for i in range(num_classes):
        best_f1 = 0
        best_thresh = 0.5 # Default starting point

        # Test a range of thresholds from 0.01 to 0.99
        thresholds = np.arange(0.01, 1.00, 0.01)

        for thresh in thresholds:
            # Convert probabilities for this class to binary predictions using the current threshold
            y_pred_bin = (y_pred_proba[:, i] > thresh).astype(int)

            # Calculate F1 score using micro-average for this single class
            # Note: For single-class optimization, the 'micro' average is the most straightforward
            current_f1 = f1_score(y_true[:, i], y_pred_bin, average='binary')

            if current_f1 > best_f1:
                best_f1 = current_f1
                best_thresh = thresh

        optimal_thresholds.append(best_thresh)
        #Optional: print(f"Class {i}: Best F1 = {best_f1:.4f}, Thresh = {best_thresh:.2f}")

    return np.array(optimal_thresholds)

In [ ]:
#FUNCTION TO EVALUATE THE FINAL TEST SET
def evaluate_test_set(model, test_data_loader, is_multi_class):
    """
    Evaluates the model on the final 10% held-out test set.
    Returns the predictions and true targets as NumPy arrays.
    """
    model.eval()
    test_outputs = []
    test_targets = []

    # NOTE: VALID_BATCH_SIZE is used here, but we will define it from global BATCH_SIZE

    with torch.no_grad():
        for data in test_data_loader:
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            # Ensure targets retain their original long dtype from CustomDataset
            targets = data['targets'].to(device, dtype=torch.long) # Corrected dtype

            outputs = model(ids, mask, token_type_ids)

            # --- UPDATED EVALUATION BLOCK ---
            if is_multi_class:
                # Multi-Class: Use Softmax so all probabilities sum to 1
                probabilities = torch.softmax(outputs, dim=1)
            else:
                # Multi-Label: Use Sigmoid for independent probabilities
                probabilities = torch.sigmoid(outputs)

            test_outputs.extend(probabilities.cpu().detach().numpy().tolist())
            # targets are 1D class indices, which is what classification_report expects for multi-class
            test_targets.extend(targets.cpu().detach().numpy().tolist())

    return np.array(test_targets), np.array(test_outputs)

# Start Train

In [ ]:
from sklearn.metrics import f1_score
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, random_split, Subset
import json # Added back the json import

# Define a path for our tuning registry
TUNING_REGISTRY_PATH = os.path.join(BEST_MODEL_DIR, "tuning_results.json")

# --- 1. HYPERPARAMETER TUNING FUNCTION (50% Subset) ---

def run_quick_tuning_run(full_df, test_df, model_key, num_labels, is_multi_class,
                         max_len, batch_size, learning_rate, epochs, patience, seed):

    # Use the 50% Subset for quick search
    train_loader_quick, val_loader_quick, _ = create_datasets_and_loaders(
        full_df,
        test_df,
        tokenizer,
        max_len,
        batch_size,
        subset_for_tuning=True, # CRITICAL: This activates the 50% rule
        seed=GLOBAL_SEED # Pass the seed to create_datasets_and_loaders
    )

    # Re-initialize Model and Optimizer
    model, _ = load_model_and_tokenizer(model_key, num_labels=num_labels, device=device)
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=learning_rate)

    # Define unique paths for this tuning run
    base_name = f"TUNING_{model_key}_LR{learning_rate}"
    ckpt_path = os.path.join(CKPT_DIR, f"ckpth_{base_name}.pth")
    best_model_path = os.path.join(BEST_MODEL_DIR, f"best_model_{base_name}.pth")

    # Define param_key unconditionally here
    param_key = f"{model_key}_LR{learning_rate}_EP{epochs}" # Moved definition here

    # Check the Registry First (Instant Load)
    if os.path.exists(TUNING_REGISTRY_PATH):
        with open(TUNING_REGISTRY_PATH, 'r') as f:
            registry = json.load(f)

        if param_key in registry:
            print(f"[REGISTRY] Found saved F1 for {param_key}: {registry[param_key]:.4f}")
            return registry[param_key]

    # LOAD MODEL CHECK (Refined logic to handle missing macro_f1 more robustly)
    if os.path.exists(best_model_path):
        print(f"\n[FOUND] Existing file for LR: {learning_rate}. Checking contents...")
        try:
            checkpoint = torch.load(best_model_path, map_location=device)

            # Initialize variables
            state_dict = None
            macro_f1 = None

            if isinstance(checkpoint, dict):
                state_dict = checkpoint.get('state_dict') or \
                             checkpoint.get('model_state_dict') or \
                             checkpoint.get('model')
                macro_f1 = checkpoint.get('macro_f1')

                if state_dict is None and any('.weight' in k for k in checkpoint.keys()):
                    state_dict = checkpoint
            else:
                state_dict = checkpoint

            # --- DECIDE: RETURN SCORE OR RE-VALIDATE ---
            if macro_f1 is not None:
                print(f"Returning saved F1 Score: {macro_f1:.4f}")
                return macro_f1

            elif state_dict is not None:
                print("Model weights found, but score missing. Re-validating now...")
                model, _ = load_model_and_tokenizer(model_key, num_labels=num_labels, device=device)
                model.load_state_dict(state_dict)
                model.to(device)

                # Get the targets/outputs from the model by running validation
                _, val_loader_quick, _ = create_datasets_and_loaders(
                    full_df, test_df, tokenizer, max_len, batch_size, subset_for_tuning=True, seed=seed
                )
                val_targets_np, val_outputs_np = evaluate_test_set(model, val_loader_quick)

                # Calculate F1 and proceed to save it properly
                if is_multi_class:
                    val_predictions = np.argmax(val_outputs_np, axis=1)
                    val_targets_indices = np.argmax(val_targets_np, axis=1)
                    macro_f1 = f1_score(val_targets_indices, val_predictions, average='macro')
                else:
                    optimal_thresholds = calculate_adaptive_thresholds(val_targets_np, val_outputs_np)
                    val_predictions_bin = (val_outputs_np > optimal_thresholds)
                    macro_f1 = f1_score(val_targets_np, val_predictions_bin, average='macro')

                # Update the checkpoint with the calculated macro_f1
                checkpoint['macro_f1'] = macro_f1
                torch.save(checkpoint, best_model_path)
                print(f"Re-validated and saved F1: {macro_f1:.4f}")
                return macro_f1
            else:
                print("[WARN] File exists but contains no recognizable weights. Forcing re-train.")
        except Exception as e:
            print(f"ERROR loading or processing checkpoint for LR: {learning_rate}: {e}. Forcing re-train.")


    print(f"\n--- Starting Tuning Run for LR: {learning_rate} ---")

    # Train the model (passing the correct IS_MULTI_CLASS flag)
    trained_model, val_targets_np, val_outputs_np = train_model(
        epochs, train_loader_quick, val_loader_quick, model,
        optimizer, ckpt_path, best_model_path, patience,
        IS_MULTI_CLASS=is_multi_class
    )

    # Calculate Final Validation Metric
    if is_multi_class:
        # Multi-Class: Convert outputs/targets to class indices and calculate F1
        val_predictions = np.argmax(val_outputs_np, axis=1)
        val_targets_indices = np.argmax(val_targets_np, axis=1)
        macro_f1 = f1_score(val_targets_indices, val_predictions, average='macro')
    else:
        # Multi-Label: Calculate Adaptive Thresholds and then F1
        optimal_thresholds = calculate_adaptive_thresholds(val_targets_np, val_outputs_np)
        val_predictions_bin = (val_outputs_np > optimal_thresholds)
        macro_f1 = f1_score(val_targets_np, val_predictions_bin, average='macro')

    # This ensures the 'return' at the top of the function has data to read next time
    checkpoint = torch.load(best_model_path)
    checkpoint['macro_f1'] = macro_f1
    torch.save(checkpoint, best_model_path)

    # Load existing, update, and save back
    current_registry = {}
    if os.path.exists(TUNING_REGISTRY_PATH):
        with open(TUNING_REGISTRY_PATH, 'r') as f:
            current_registry = json.load(f)

    current_registry[param_key] = float(macro_f1)

    with open(TUNING_REGISTRY_PATH, 'w') as f:
        json.dump(current_registry, f)
    # ---------------------------------------

    print(f"Tuning Result (Val Macro F1): {macro_f1:.4f}")
    return macro_f1


#EXECUTION BLOCK: SEARCH FOR BEST LR

# Define the Learning Rates to test
LEARNING_RATE_CANDIDATES = [5e-5, 2e-5, 3e-5] # Test common ranges
results = {}

# Use the constants defined earlier in your notebook
TUNE_EPOCHS = 3 # Use fewer epochs for faster tuning (or match your original EPOCHS)

print("\n=====================================================")
print(f"STARTING HYPERPARAMETER TUNING on 50% SUBSET")
print("=====================================================")

for lr in LEARNING_RATE_CANDIDATES:
    f1 = run_quick_tuning_run(
        full_df=FULL_TRAIN_VAL_DF,
        test_df=TEST_DF,
        model_key=CURRENT_MODEL_KEY,
        num_labels=NUM_LABELS_STAGE,
        is_multi_class=IS_MULTI_CLASS,
        max_len=MAX_LEN,
        batch_size=BATCH_SIZE,
        learning_rate=lr,
        epochs=TUNE_EPOCHS,
        patience=PATIENCE,
        seed=GLOBAL_SEED
    )
    results[lr] = f1


# Find the best learning rate
best_lr = max(results, key=results.get)
best_f1 = results[best_lr]

print("\n================== TUNING SUMMARY ==================")
print(f"Results: {results}")
print(f"BEST LEARNING RATE: {best_lr} (Validation Macro F1: {best_f1:.4f})")
print("=====================================================")

# Store the best LR to use for the final K-Fold run
BEST_LEARNING_RATE_FOUND = best_lr


#Saving the best learning rate (hyperparameter)

In [ ]:
print("\n================== TUNING SUMMARY ==================")
print(f"BEST LEARNING RATE: {best_lr} (Validation Macro F1: {best_f1:.4f})")
print("=====================================================")

# --- EXPORT THE HYPERPARAMETER TO A FILE (NEW) ---
HYPERPARAM_DIR = "/content/gdrive/MyDrive/Hyperparams"
os.makedirs(HYPERPARAM_DIR, exist_ok=True)
FILE_PATH = os.path.join(HYPERPARAM_DIR, "best_lr_cryptobert_stage1.txt")

with open(FILE_PATH, 'w') as f:
    f.write(str(best_lr))

print(f"\nSUCCESS: Best Learning Rate exported to {FILE_PATH}")
# END OF NOTEBOOK 1

#Training & Validating Using K-Fold


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import DataLoader, Subset
import pandas as pd
import numpy as np
import torch
import os
from typing import Tuple

# NOTE: Ensure all prerequisites are defined globally or imported:
# CustomDataset, load_model_and_tokenizer, train_model,
# calculate_adaptive_thresholds, evaluate_test_set, target_list

def run_kfold_experiment(full_df: pd.DataFrame,
                         test_df: pd.DataFrame,
                         model_key: str,
                         num_labels: int,
                         is_multi_class: bool,
                         k_folds: int = 5,
                         epochs: int = EPOCHS,
                         patience: int = PATIENCE,
                         learning_rate: float = None,
                         batch_size: int = BATCH_SIZE,
                         seed: int = GLOBAL_SEED
                         ) -> Tuple[float, float]:

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=seed)
    fold_metrics = []
    all_reports = []

    full_dataset = CustomDataset(full_df, tokenizer, MAX_LEN)
    test_dataset_obj = CustomDataset(test_df, tokenizer, MAX_LEN)
    test_data_loader = DataLoader(test_dataset_obj, batch_size=batch_size, shuffle=False)

    print(f"--- Starting {k_folds}-Fold CV for {model_key} (LR: {learning_rate}) ---")

    for fold, (train_index, val_index) in enumerate(kf.split(full_df)):
        base_name = f"{model_key}_FOLD{fold+1}_LR{learning_rate}"
        best_model_path = os.path.join(BEST_MODEL_DIR, f"best_model_{base_name}.pth")

        current_fold_model = None # Variable to hold the model for the current fold
        val_targets_np_for_threshold = None # For adaptive thresholds if multi-label
        val_outputs_np_for_threshold = None # For adaptive thresholds if multi-label

        # --- RESUME LOGIC ---
        if os.path.exists(best_model_path):
            print(f"⏩ Fold {fold+1} already exists. Loading model...")
            try:
                checkpoint = torch.load(best_model_path, map_location=device, weights_only=False)
                current_fold_model, _ = load_model_and_tokenizer(model_key, num_labels=num_labels, device=device)
                current_fold_model.load_state_dict(checkpoint['state_dict'])
                current_fold_model.eval()

                # Get validation set predictions for potential adaptive thresholds
                val_subset = Subset(full_dataset, val_index)
                val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
                val_targets_np_for_threshold, val_outputs_np_for_threshold = evaluate_test_set(current_fold_model, val_loader, is_multi_class)

            except Exception as e:
                print(f"❌ Failed to load model or metrics for Fold {fold+1}: {e}. Proceeding to train.")
                current_fold_model = None # Set to None to trigger training path

        # --- TRAINING SECTION (Only runs if model was not successfully loaded) ---
        if current_fold_model is None:
            print(f"\n====================== FOLD {fold+1}/{k_folds} ======================")
            train_subset = Subset(full_dataset, train_index)
            val_subset = Subset(full_dataset, val_index)
            train_data_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
            val_data_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

            model_for_training, _ = load_model_and_tokenizer(model_key, num_labels=num_labels, device=device)
            optimizer = torch.optim.AdamW(params=model_for_training.parameters(), lr=learning_rate)

            # train_model returns the trained model and val data used for threshold calculation
            current_fold_model, val_targets_np_for_threshold, val_outputs_np_for_threshold = train_model(
                epochs, train_data_loader, val_data_loader, model_for_training,
                optimizer, os.path.join(CKPT_DIR, f"ckpth_{base_name}.pth"),
                best_model_path, patience, IS_MULTI_CLASS=is_multi_class
            )

        # --- ENSURE TEST SET EVALUATION FOR FINAL REPORTING ---
        # This block is executed for every fold, after current_fold_model is either loaded or trained.
        test_targets_np, test_outputs_np = evaluate_test_set(current_fold_model, test_data_loader, is_multi_class)

        # 6. Evaluation Logic using the obtained test_targets_np and test_outputs_np
        test_predictions_bin = None
        y_true_eval = None

        if is_multi_class:
            # For multi-class, test_targets_np is 1D (class indices) and test_outputs_np is 2D (probabilities)
            test_predictions_bin = np.argmax(test_outputs_np, axis=1)
            y_true_eval = test_targets_np # Already 1D class indices
        else:
            # Multi-Label case (requires val_targets_np_for_threshold and val_outputs_np_for_threshold)
            if val_targets_np_for_threshold is None or val_outputs_np_for_threshold is None:
                 print("WARNING: Validation data not available for adaptive thresholds in multi-label mode. Using default threshold 0.5.")
                 optimal_thresholds = np.full(num_labels, 0.5) # Default to 0.5 for all labels
            else:
                optimal_thresholds = calculate_adaptive_thresholds(val_targets_np_for_threshold, val_outputs_np_for_threshold)

            test_predictions_bin = (test_outputs_np > optimal_thresholds).astype(int)
            y_true_eval = test_targets_np # Still 1D class indices from the dataset, for multi-label report it might need one-hot

        # Generate report
        report_dict = classification_report(
            y_true_eval,
            test_predictions_bin,
            target_names=target_list,
            output_dict=True,
            zero_division=0
        )
        all_reports.append(report_dict)
        fold_metrics.append(report_dict['macro avg']['f1-score'])
        print(f"✅ Fold {fold+1} metrics calculated. Macro F1: {report_dict['macro avg']['f1-score']:.4f}")

    # 7. Final Aggregation and Reporting (After the Loop)
    mean_f1 = np.mean(fold_metrics)
    std_f1 = np.std(fold_metrics)
    avg_report = {}

    # Define which labels and averages to include
    labels_to_average = target_list + ['macro avg', 'weighted avg']

    for label in labels_to_average:
        avg_report[label] = {}
        # We now loop through all three core metrics
        for metric in ['precision', 'recall', 'f1-score']:
            # Collect the metric value for this label from all 5 reports
            metric_values = [report[label][metric] for report in all_reports if label in report]
            # Average the metric across the 5 folds
            avg_report[label][metric] = np.mean(metric_values) if metric_values else 0.0

    # 8. Print the Final Averaged Classification Report
    print("\n====================== AVERAGED PER-LABEL REPORT ======================")
    print(f"Metrics are averaged across all {k_folds} K-Fold runs on the 10% Test Set:")

    for label in target_list:
        print(f"\n--- {label} ---")
        print(f"  Precision: {avg_report[label]['precision']:.4f}")
        print(f"  Recall:    {avg_report[label]['recall']:.4f}")
        print(f"  F1-Score:  {avg_report[label]['f1-score']:.4f}")

    # NEW: Expanded Macro Average Display
    print(f"\nMACRO AVERAGE (Final Reported Metrics):")
    print(f"  Precision: {avg_report['macro avg']['precision']:.4f}")
    print(f"  Recall:    {avg_report['macro avg']['recall']:.4f}")
    print(f"  F1-Score:  {avg_report['macro avg']['f1-score']:.4f} \u00b1 {std_f1:.4f}")
    print("=====================================================================")

    return avg_report['macro avg']['f1-score'], std_f1

In [ ]:
# --- EXECUTION BLOCK ---
print("Checking variables before calling...")
FULL_TRAIN_VAL_DF = FULL_TRAIN_VAL_DF.reset_index(drop=True)

print(f"Dataframe rows: {len(FULL_TRAIN_VAL_DF)}")
print("Attempting to call run_kfold_experiment now...")

final_f1, final_std = run_kfold_experiment(
    full_df=FULL_TRAIN_VAL_DF,
    test_df=TEST_DF,
    model_key=CURRENT_MODEL_KEY,
    num_labels=NUM_LABELS_STAGE,
    is_multi_class=IS_MULTI_CLASS,
    k_folds=5,
    epochs=EPOCHS,
    patience=PATIENCE,
    learning_rate=BEST_LEARNING_RATE_FOUND,
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)

print("K-Fold Experiment Complete!")

# Test with New Input Text

In [ ]:
def classify_text(model, text, tokenizer, max_len, threshold=0.5):
    # Prepare the text
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)

    # Get the model outputs
    with torch.no_grad():
        outputs = model(input_ids, attention_mask, token_type_ids)

    # Convert to probabilities
    probabilities = torch.sigmoid(outputs).cpu().detach().numpy().tolist()

    # Define the class labels in the same order that the model was trained on
    class_labels = ['religion_creed', 'race_ethnicity', 'physical_disability', 'gender_sexualorientation', 'other']


    # Convert the probabilities to labels
    predicted_labels = [class_labels[i] for i, prob in enumerate(probabilities[0]) if prob > threshold]

    return probabilities, predicted_labels


In [ ]:
text = ""
probabilities, predicted_labels = classify_text(model, text, tokenizer, MAX_LEN)
print("Probabilities:", probabilities)
print("Predicted labels:", predicted_labels)

In [ ]:
text = ""
probabilities, predicted_labels = classify_text(model, text, tokenizer, MAX_LEN)
print("Probabilities:", probabilities)
print("Predicted labels:", predicted_labels)

In [ ]:
text = ""
probabilities, predicted_labels = classify_text(model, text, tokenizer, MAX_LEN)
print("Probabilities:", probabilities)
print("Predicted labels:", predicted_labels)